<a href="https://colab.research.google.com/github/Namesakenberg/deep_learning/blob/main/Next_word_prediction_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [104]:
with open('/content/alice_in_wonderland.txt','r') as file:
  content = file.read()


In [105]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts((content.split('\n')))

In [106]:
for sentence in content.split('\n'):
  tokenizer.texts_to_sequences([sentence])[0]

In [107]:
# create the training data
vectors = []
for sentence in content.split('\n'):
   vectors.append(tokenizer.texts_to_sequences([sentence])[0])

In [108]:
inputs = []
outputs = []

for sentence in vectors:
  if len(sentence)>1:
    for i in range(len(sentence)-1):
      inputs.append(sentence[:i+1])
      outputs.append(sentence[i+1])

In [109]:
df = pd.DataFrame(
    {"sentence":inputs,
    "next_word":outputs}
)

In [110]:
df.head()

,sentence,next_word
0,[278],423
1,"[278, 423]",12
2,"[278, 423, 12]",699
3,[278],423
4,"[278, 423]",12


In [111]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

inputs = pad_sequences(
    inputs,
    maxlen=max(len(seq) for seq in inputs),
    padding='post',
)


In [112]:
inputs.shape

(25073, 15)

In [113]:
outputs = np.array(outputs)

In [114]:
print(type(inputs))
print(type(outputs))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [115]:
vocab_size = len(tokenizer.word_index) + 1
print("Tokenizer vocabulary size:", vocab_size)

unique_targets = len(set(outputs))
print("Unique output classes:", unique_targets)

Tokenizer vocabulary size: 2647
Unique output classes: 2504


In [116]:
print(len(outputs))

25073


In [117]:
print("Min label:", outputs.min())
print("Max label:", outputs.max())

Min label: 1
Max label: 2646


In [118]:
from tensorflow.keras.utils import to_categorical
outputs_ohe = to_categorical(outputs , num_classes=2647)

In [119]:
print(inputs.shape)
print(outputs.shape)

(25073, 15)
(25073,)


In [ ]:
# architecture

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten,LSTM

model = Sequential()
model.add(Embedding(input_dim = 2647,output_dim = 128,input_length = 15))
model.add(LSTM(150))
model.add(Dense(2647,activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(loss='categorical_crossentropy' , optimizer='adam' , metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(inputs,outputs_ohe ,epochs=100)

Epoch 1/100
784/784 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.0549 - loss: 6.3628
Epoch 2/100
784/784 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.0610 - loss: 5.9534
Epoch 3/100
784/784 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.0598 - loss: 5.9388
Epoch 4/100
784/784 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.0642 - loss: 5.9100
Epoch 5/100
784/784 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.0600 - loss: 5.9278
Epoch 6/100
784/784 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.0621 - loss: 5.9059
Epoch 7/100
784/784 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.0609 - loss: 5.8925
Epoch 8/100
784/784 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.0603 - loss: 5.8775
Epoch 9/100
784/784 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.0643 - loss: 5.7646
Epoch 10/100
784/784 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.0651 - loss: 5.5831
Epoch 11/100
784/784 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.0779 - loss: 5.4507
Epoch 12/100
784/784 ━━━━━━━━━━━━━━━

In [120]:
def testing(test_word):
  test_word_tokenized=tokenizer.texts_to_sequences([test_word])[0]
  padded_test_word = pad_sequences([test_word_tokenized],maxlen =max(len(seq) for seq in inputs),padding='post')

  pred_prob = model.predict(padded_test_word)
  pred_id = np.argmax(pred_prob)


  for key,value in tokenizer.word_index.items():
    if value == pred_id:
      return key

In [121]:
test_sentences=[
    "alice was beginning to get very",

"the rabbit hole went straight on like a",

"she was considering in her own mind",

"the white rabbit was still in sight hurrying down",

"suddenly alice found herself falling down a very",

"the caterpillar and alice looked at each",

"the queen of hearts shouted at the top of her",

"cheshire cat appeared in the branches of a",

"alice opened the door and saw a beautiful",

"the mad hatter was sitting at a large",
]

In [122]:
for i in test_sentences:
  print("input : ",i)
  print("next_word : ",testing(i))

input :  alice was beginning to get very
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
next_word :  tired
input :  the rabbit hole went straight on like a
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
next_word :  tunnel
input :  she was considering in her own mind
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
next_word :  sobbing
input :  the white rabbit was still in sight hurrying down
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
next_word :  the
input :  suddenly alice found herself falling down a very
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
next_word :  nervous
input :  the caterpillar and alice looked at each
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
next_word :  other
input :  the queen of hearts shouted at the top of her
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
next_word :  mouth
input :  cheshire cat appeared in the branches of a
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
next_word :  low
input :  alice opened the door and saw a beautiful
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
next_word :  alice
input :  the mad hatter wa